In [ ]:
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.visualization
import named_arrays as na
import optika

In [ ]:
radius_aperture = 100 * u.mm

In [ ]:
f_number = 10

So the focal length of the primary is then

In [ ]:
focal_length = f_number * radius_aperture
focal_length

In [ ]:
sag_primary = optika.sags.ParabolicSag(-focal_length)

In [ ]:
material_primary = optika.materials.Mirror()

In [ ]:
aperture_primary = optika.apertures.CircularAperture(radius_aperture)

In [ ]:
translation_primary = na.transformations.Cartesian3dTranslation(z=focal_length)

In [ ]:
angle_primary = 5 * u.deg
rotation_primary = na.transformations.Cartesian3dRotationX(angle_primary)

In [ ]:
transformation_primary = translation_primary @ rotation_primary

In [ ]:
primary = optika.surfaces.Surface(
    name="primary",
    sag=sag_primary,
    material=material_primary,
    aperture=aperture_primary,
    transformation=transformation_primary,
    is_pupil_stop=True,
)

In [ ]:
width_pixel = 15 * u.um

In [ ]:
num_pixel = na.Cartesian2dVectorArray(256, 256)

In [ ]:
axis_pixel = na.Cartesian2dVectorArray("detector_x", "detector_y")

In [ ]:
rotation_flip = na.transformations.Cartesian3dRotationX(180 * u.deg)
rotation_sensor = rotation_primary @ rotation_flip @ rotation_primary
transformation_sensor_axis = translation_primary @ rotation_sensor
transformation_sensor = transformation_sensor_axis @ translation_primary

In [ ]:
sensor = optika.sensors.ImagingSensor(
    name="sensor",
    width_pixel=width_pixel,
    axis_pixel=axis_pixel,
    num_pixel=num_pixel,
    transformation=transformation_sensor,
    is_field_stop=True,
)

In [ ]:
pupil = na.Cartesian2dVectorLinearSpace(
    start=-1,
    stop=1,
    axis=na.Cartesian2dVectorArray("px", "py"),
    num=5,
    centers=True,
)

In [ ]:
field = na.Cartesian2dVectorLinearSpace(
    start=-1,
    stop=1,
    axis=na.Cartesian2dVectorArray("fx", "fy"),
    num=5,
    centers=True,
)

In [ ]:
wavelength = 500 * u.nm

In [ ]:
grid_input = optika.vectors.ObjectVectorArray(
    wavelength=wavelength,
    field=field,
    pupil=pupil,
)

In [ ]:
system = optika.systems.SequentialSystem(
    surfaces=[
        primary,
    ],
    sensor=sensor,
    grid_input=grid_input,
)

In [ ]:
# plot the system
with astropy.visualization.quantity_support():
    fig, ax = plt.subplots(constrained_layout=True)
    ax.set_aspect("equal")
    system.plot(
        ax=ax,
        components=("z", "y"),
        kwargs_rays=dict(
            color="tab:blue",
        ),
        color="black",
        zorder=10,
    )

In [ ]:
position = system.rayfunction_default.outputs.position

In [ ]:
position_relative = position - position.mean(pupil.axes)

In [ ]:
with astropy.visualization.quantity_support():
    fig, ax = na.plt.subplots(
        axis_rows=field.axis.x,
        axis_cols=field.axis.y,
        nrows=field.num,
        ncols=field.num,
        sharex=True,
        sharey=True,
        constrained_layout=True,
    )
    na.plt.scatter(
        position_relative.x,
        position_relative.y,
        ax=ax,
    )